In [104]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import KNNImputer

# load data

In [105]:
# Load pandas dataframes from pkl files
all_depression_train = pd.read_pickle("data/train.pkl")
all_depression_test = pd.read_pickle("data/test.pkl")

In [106]:
# columns with polygenetic risk scores (PRS) from UK Biobank
PRS_columns = [
    "p26202", "p26203", "p26204", "p26205", "p26206", "p26207", "p26208", "p26209", 
    "p26210", "p26211", "p26212", "p26213", "p26214", "p26215", "p26216", "p26217", 
    "p26218", "p26219", "p26220", "p26221", "p26222", "p26223", "p26224", "p26225", 
    "p26226", "p26227", "p26228", "p26229", "p26231", "p26232", "p26233", "p26234", 
    "p26235", "p26236", "p26237", "p26238", "p26239", "p26240", "p26241", "p26242", 
    "p26243", "p26244", "p26245", "p26246", "p26247", "p26248", "p26249", "p26250", 
    "p26251", "p26252", "p26253", "p26254", "p26255", "p26256", "p26257", "p26258", 
    "p26259", "p26260", "p26261", "p26262", "p26263", "p26264", "p26265", "p26266", 
    "p26267", "p26268", "p26269", "p26270", "p26271", "p26272", "p26273", "p26274", 
    "p26275", "p26276", "p26277", "p26278", "p26279", "p26280", "p26281", "p26282", 
    "p26283", "p26284", "p26285", "p26286", "p26287", "p26289", "p26290"
]

In [107]:
# Check missing values and percentages for all columns in PRS_columns
def check_missing_values(df, columns):
    # Total number of rows in the DataFrame
    total_rows = df.shape[0]
    
    # Iterate over each column in the PRS_columns list
    for column in columns:
        if column in df.columns:
            # Check the number of missing values
            missing_values = df[column].isnull().sum()
            
            # Calculate the percentage of missing values
            missing_percentage = (missing_values / total_rows) * 100
            
            # Print the results for the column
            print(f"Column: {column}")
            print(f"Missing values: {missing_values}")
            print(f"Percentage of missing values: {missing_percentage:.2f}%\n")
        else:
            print(f"Column: {column} is not found in the DataFrame.\n")

# Apply the function to the all_depression_train dataset
check_missing_values(all_depression_train, PRS_columns)

Column: p26202
Missing values: 0
Percentage of missing values: 0.00%

Column: p26203
Missing values: 0
Percentage of missing values: 0.00%

Column: p26204
Missing values: 0
Percentage of missing values: 0.00%

Column: p26205
Missing values: 0
Percentage of missing values: 0.00%

Column: p26206
Missing values: 0
Percentage of missing values: 0.00%

Column: p26207
Missing values: 0
Percentage of missing values: 0.00%

Column: p26208
Missing values: 0
Percentage of missing values: 0.00%

Column: p26209
Missing values: 0
Percentage of missing values: 0.00%

Column: p26210
Missing values: 0
Percentage of missing values: 0.00%

Column: p26211
Missing values: 0
Percentage of missing values: 0.00%

Column: p26212
Missing values: 0
Percentage of missing values: 0.00%

Column: p26213
Missing values: 0
Percentage of missing values: 0.00%

Column: p26214
Missing values: 0
Percentage of missing values: 0.00%

Column: p26215
Missing values: 0
Percentage of missing values: 0.00%

Column: p26216
Missi

## define dataset size

In [108]:
# define size of each dataset
all_depression_train_size = 500
all_depression_test_size = 500

# randomly sample to the desired dataset sizes
# Downsample datasets
all_depression_train = all_depression_train.sample(all_depression_train_size, random_state=81)
all_depression_test = all_depression_test.sample(all_depression_test_size, random_state=81)

In [109]:
# Print the number of rows and balance for each dataset
def print_dataset_info(dataset, dataset_name):
    print(f"{dataset_name} - Number of rows: {len(dataset)}")
    print(dataset['target'].value_counts(normalize=True) * 100)
    print("\n")

# Print information for each dataset
print_dataset_info(all_depression_train, "all_depression_train")
print_dataset_info(all_depression_test, "all_depression_test")

all_depression_train - Number of rows: 500
0    50.2
1    49.8
Name: target, dtype: float64


all_depression_test - Number of rows: 500
0    51.2
1    48.8
Name: target, dtype: float64




## descriptives of the two datasets

In [110]:
# filter the following columns from train and test dataset
# p31 - sex
# age_at_baseline
# target - rename to depression
# p130895 - rename to source_of_depression_reporting
# new column - p53_i0 minus p130894 - name months_depression_after_baseline

# For training data
train_descriptives = all_depression_train[['p31', 'age_at_baseline', 'target', 'p130895', 'p53_i0', 'p130894']].copy()
train_descriptives.rename(columns={
    'p31': 'sex',
    'target': 'depression',
    'p130895': 'source_of_depression_reporting'
}, inplace=True)

# Ensure the columns are in datetime format
train_descriptives['p53_i0'] = pd.to_datetime(train_descriptives['p53_i0'])
train_descriptives['p130894'] = pd.to_datetime(train_descriptives['p130894'])

# Calculate the difference in days, convert to months, and convert to int
train_descriptives['months_depression_after_baseline'] = (train_descriptives['p130894'] - train_descriptives['p53_i0']).dt.days / 30

# Drop the unnecessary columns after calculation
train_descriptives.drop(columns=['p53_i0', 'p130894'], inplace=True)

# For testing data
test_descriptives = all_depression_test[['p31', 'age_at_baseline', 'target', 'p130895', 'p53_i0', 'p130894']].copy()
test_descriptives.rename(columns={
    'p31': 'sex',
    'target': 'depression',
    'p130895': 'source_of_depression_reporting'
}, inplace=True)

# Ensure the columns are in datetime format
test_descriptives['p53_i0'] = pd.to_datetime(test_descriptives['p53_i0'])
test_descriptives['p130894'] = pd.to_datetime(test_descriptives['p130894'])

# Calculate the difference in days, convert to months, and convert to int
test_descriptives['months_depression_after_baseline'] = (test_descriptives['p130894'] - test_descriptives['p53_i0']).dt.days / 30

# Drop the unnecessary columns after calculation
test_descriptives.drop(columns=['p53_i0', 'p130894'], inplace=True)

# Print out the first few rows for both dataframes
print(train_descriptives.head())
print(test_descriptives.head())

            sex  age_at_baseline  depression source_of_depression_reporting  \
126  Category_A               68           1                     Category_E   
314  Category_A               55           0                     Category_E   
267  Category_A               63           1                     Category_E   
282  Category_A               56           0                     Category_B   
496  Category_E               55           0                     Category_C   

     months_depression_after_baseline  
126                         95.933333  
314                         51.100000  
267                        -30.833333  
282                        143.966667  
496                         58.600000  
            sex  age_at_baseline  depression source_of_depression_reporting  \
126  Category_E               51           0                     Category_C   
314  Category_D               49           0                     Category_D   
267  Category_C               68           0    

In [111]:
# Filter rows where depression == 1 for train data
train_depression_rows = train_descriptives[train_descriptives['depression'] == 1]

# Calculate min and max for months_depression_after_baseline in train data
train_min_months = train_depression_rows['months_depression_after_baseline'].min()
train_max_months = train_depression_rows['months_depression_after_baseline'].max()

print(f"Train data - Min months: {train_min_months}, Max months: {train_max_months}")

# Filter rows where depression == 1 for test data
test_depression_rows = test_descriptives[test_descriptives['depression'] == 1]

# Calculate min and max for months_depression_after_baseline in test data
test_min_months = test_depression_rows['months_depression_after_baseline'].min()
test_max_months = test_depression_rows['months_depression_after_baseline'].max()

print(f"Test data - Min months: {test_min_months}, Max months: {test_max_months}")

Train data - Min months: -49.333333333333336, Max months: 195.6
Test data - Min months: -50.46666666666667, Max months: 199.0


In [112]:
# Define the mapping for values in source_of_depression_reporting
replace_mapping = {
    'Self-report only': 'self-report',
    'Self-report and other source(s)': 'self-report',
    'Primary care only': 'primary care',
    'Primary care and other source(s)': 'primary care',
    'Hospital admissions data only': 'hospital admission',
    'Hospital admissions data and other source(s)': 'hospital admission',
    'Death register only': 'hospital admission'
}

# Apply the mapping to train data
train_descriptives['source_of_depression_reporting'] = train_descriptives['source_of_depression_reporting'].replace(replace_mapping)

# Apply the mapping to test data
test_descriptives['source_of_depression_reporting'] = test_descriptives['source_of_depression_reporting'].replace(replace_mapping)

# Print the updated unique values to verify the changes
print("Updated unique values in source_of_depression_reporting (train):")
print(train_descriptives['source_of_depression_reporting'].unique())

print("Updated unique values in source_of_depression_reporting (test):")
print(test_descriptives['source_of_depression_reporting'].unique())

Updated unique values in source_of_depression_reporting (train):
['Category_E' 'Category_B' 'Category_C' 'Category_D' 'Category_A']
Updated unique values in source_of_depression_reporting (test):
['Category_C' 'Category_D' 'Category_A' 'Category_B' 'Category_E']


In [113]:
# Function to calculate the descriptive statistics
def create_summary(df):
    summary = pd.DataFrame()

    # Sex (empty row)
    summary.loc['sex', 'percentage'] = ''
    summary.loc['Female', 'percentage'] = df['sex'].value_counts(normalize=True).get('Female', 0) * 100
    summary.loc['Male', 'percentage'] = df['sex'].value_counts(normalize=True).get('Male', 0) * 100

    # Age at baseline (mean and standard deviation)
    summary.loc['age_at_baseline', 'mean'] = df['age_at_baseline'].mean()
    summary.loc['age_at_baseline', 'standard deviation'] = df['age_at_baseline'].std()

    # Depression (empty row)
    summary.loc['depression', 'percentage'] = ''
    summary.loc['Yes', 'percentage'] = df['depression'].value_counts(normalize=True).get(1, 0) * 100
    summary.loc['No', 'percentage'] = df['depression'].value_counts(normalize=True).get(0, 0) * 100

    # Months depression after baseline (mean and standard deviation, only for rows with depression == 1)
    depression_rows = df[df['depression'] == 1]
    summary.loc['months_depression_after_baseline', 'mean'] = depression_rows['months_depression_after_baseline'].mean()
    summary.loc['months_depression_after_baseline', 'standard deviation'] = depression_rows['months_depression_after_baseline'].std()

    # Source of reporting (empty row)
    summary.loc['source_of_depression_reporting', 'percentage'] = ''
    # Categories of source_of_depression_reporting with percentages
    for category in df['source_of_depression_reporting'].unique():
        if category is not None:
            summary.loc[category, 'percentage'] = df['source_of_depression_reporting'].value_counts(normalize=True).get(category, 0) * 100

    return summary

# Calculate the descriptive statistics for train and test data
train_summary = create_summary(train_descriptives)
test_summary = create_summary(test_descriptives)

# Save the descriptives to CSV
train_summary.to_csv('data/train_descriptives_summary.csv')
test_summary.to_csv('data/test_descriptives_summary.csv')

In [114]:
train_summary.head(20)

,percentage,mean,standard deviation
sex,,NaN,NaN
Female,0,NaN,NaN
Male,0,NaN,NaN
age_at_baseline,NaN,54.778000,8.901190
depression,,NaN,NaN
Yes,49.8,NaN,NaN
No,50.2,NaN,NaN
months_depression_after_baseline,NaN,79.195181,61.994047
source_of_depression_reporting,,NaN,NaN
Category_E,20.8,NaN,NaN


In [115]:
test_summary.head(20)

,percentage,mean,standard deviation
sex,,NaN,NaN
Female,0,NaN,NaN
Male,0,NaN,NaN
age_at_baseline,NaN,55.016000,8.785791
depression,,NaN,NaN
Yes,48.8,NaN,NaN
No,51.2,NaN,NaN
months_depression_after_baseline,NaN,70.767623,61.642417
source_of_depression_reporting,,NaN,NaN
Category_C,20.6,NaN,NaN


## define special columns

In [116]:
# special columns which will be removed from train datasets and ignored for all cleaning in test datasets
special_columns = ['eid', 'p130894', 'p130895', 'p53_i0']
# eid (participants ID)
# p130894 (date of first occurence of depression),
# p130895 (source of first occurence of depression)
# p53_i0 (date of baseline assessment)

In [117]:
# remove special columns from train dataset
all_depression_train = all_depression_train.drop(columns=special_columns)

In [118]:
# move special columns to the end
# Rearranging columns in all_depression_test
cols = [col for col in all_depression_test.columns if col not in special_columns]
all_depression_test = all_depression_test[cols + special_columns]

# clean data

In [119]:
# Function to check if a column contains list-like structures
def contains_list_structure(column):
    return column.apply(lambda x: isinstance(x, list)).any()

# Function to identify columns with list structures in a dataframe, excluding special_columns
def identify_columns_with_lists(df, special_columns):
    return df.columns[~df.columns.isin(special_columns)][df.loc[:, ~df.columns.isin(special_columns)].apply(contains_list_structure)]

# Identify columns with list structures in all datasets, excluding special_columns
columns_with_lists_all_train = identify_columns_with_lists(all_depression_train, special_columns)
columns_with_lists_all_test = identify_columns_with_lists(all_depression_test, special_columns)

In [120]:
# Function to convert list-like structures to pure strings
def convert_to_string(value):
    if isinstance(value, list):
        return ', '.join(map(str, value))  # Join list elements into a single string
    elif pd.isna(value):
        return ''  # Handle NaN values by converting them to empty strings
    else:
        return str(value)  # Convert any other type to a string

# Function to apply conversion to all object-type columns in a DataFrame, excluding special_columns
def convert_columns_to_string(df, special_columns):
    for column in df.select_dtypes(include=['object']).columns:
        if column not in special_columns:
            df[column] = df[column].apply(convert_to_string)
    return df

# Apply the conversion function to all datasets, excluding special_columns
all_depression_train = convert_columns_to_string(all_depression_train, special_columns)
all_depression_test = convert_columns_to_string(all_depression_test, special_columns)

In [121]:
# Function to convert numeric columns to float64, excluding special_columns and the 'target' column
def convert_numeric_to_float64(df, special_columns, target_column='target'):
    numeric_columns = df.select_dtypes(include=['number']).columns
    columns_to_exclude = special_columns + [target_column]
    columns_to_convert = numeric_columns.difference(columns_to_exclude)
    df[columns_to_convert] = df[columns_to_convert].astype('float64')
    return df

# Apply the conversion function to all datasets, excluding special_columns and the target column
all_depression_train = convert_numeric_to_float64(all_depression_train, special_columns)
all_depression_test = convert_numeric_to_float64(all_depression_test, special_columns)

## handle columns with many missing values

In [122]:
# Function to calculate and print missing value statistics in percentage
def calculate_missing_values_statistics(df, missing_percentage=0.10):
    # Total number of rows in the dataframe
    total_rows = len(df)
    
    # 1. Number of columns with missing values
    columns_with_missing_values = df.columns[df.isnull().sum() > 0]
    num_columns_with_missing_values = len(columns_with_missing_values)
    print(f"Number of columns with missing values: {num_columns_with_missing_values}")
    
    # 2. Mean and standard deviation of missing values per column (in percentage)
    missing_values_per_column = df.isnull().sum() / total_rows * 100  # Convert to percentage
    mean_missing_values = missing_values_per_column.mean()
    std_missing_values = missing_values_per_column.std()
    print(f"Mean of missing values per column (in %): {mean_missing_values:.2f}%")
    print(f"Standard deviation of missing values per column (in %): {std_missing_values:.2f}%")
    
    # 3. Number of columns with more than a given percentage (e.g., 10%) of missing values
    threshold = total_rows * missing_percentage
    columns_with_high_missing = df.columns[df.isnull().sum() > threshold]
    num_columns_with_high_missing = len(columns_with_high_missing)
    print(f"Number of columns with more than {missing_percentage*100}% missing values: {num_columns_with_high_missing}")
    
    return columns_with_missing_values, columns_with_high_missing

# Example usage of the function
columns_with_missing, columns_with_high_missing = calculate_missing_values_statistics(all_depression_train, missing_percentage=0.10)

Number of columns with missing values: 0
Mean of missing values per column (in %): 0.00%
Standard deviation of missing values per column (in %): 0.00%
Number of columns with more than 10.0% missing values: 0


In [123]:
# Function to drop columns with more than a certain percentage of missing values,
# excluding special columns, based only on the training datasets
def drop_columns_with_missing_values_based_on_train(train_df, test_df, special_columns, missing_percentage=0.10):
    # Calculate the threshold for dropping columns based on the training dataset
    threshold = len(train_df) * missing_percentage
    
    # Get the columns to drop based on the training data
    columns_to_drop = train_df.columns[train_df.isnull().sum() > threshold].tolist()
    
    # Ensure that special columns are not dropped
    columns_to_drop = [col for col in columns_to_drop if col not in special_columns]
    
    # Drop the columns from both training and testing datasets
    train_cleaned = train_df.drop(columns=columns_to_drop)
    test_cleaned = test_df.drop(columns=columns_to_drop)
    
    # Ensure special columns are retained
    special_columns_to_keep = [col for col in special_columns if col in train_df.columns]
    
    # Ensure special columns are at the end
    train_cleaned = pd.concat([train_cleaned, train_df[special_columns_to_keep]], axis=1)
    test_cleaned = pd.concat([test_cleaned, test_df[special_columns_to_keep]], axis=1)
    
    # Remove any duplicate columns in case they were re-added
    train_cleaned = train_cleaned.loc[:, ~train_cleaned.columns.duplicated()]
    test_cleaned = test_cleaned.loc[:, ~test_cleaned.columns.duplicated()]
    
    return train_cleaned, test_cleaned

# Apply the function to the datasets
all_depression_train_cleaned, all_depression_test_cleaned = drop_columns_with_missing_values_based_on_train(
    all_depression_train, all_depression_test, special_columns)

## impute numeric missing values

In [124]:
# Function to calculate missing value statistics for numeric columns only
def calculate_missing_values_numeric_columns(df, special_columns=None):
    if special_columns is None:
        special_columns = []
    
    # Select only the numeric columns for calculation, excluding special columns
    numeric_columns = df.select_dtypes(include=['number']).columns.difference(special_columns)
    
    # Total number of rows in the dataframe
    total_rows = len(df)
    
    # 1. Number of numeric columns with missing values
    numeric_columns_with_missing_values = numeric_columns[df[numeric_columns].isnull().sum() > 0]
    num_numeric_columns_with_missing_values = len(numeric_columns_with_missing_values)
    print(f"Number of numeric columns with missing values: {num_numeric_columns_with_missing_values}")
    
    # 2. Mean and standard deviation of missing values per numeric column (in percentage)
    missing_values_per_column_percent = df[numeric_columns].isnull().sum() / total_rows * 100  # Convert to percentage
    mean_missing_values_percent = missing_values_per_column_percent.mean()
    std_missing_values_percent = missing_values_per_column_percent.std()
    print(f"Mean of missing values per numeric column (in %): {mean_missing_values_percent:.2f}%")
    print(f"Standard deviation of missing values per numeric column (in %): {std_missing_values_percent:.2f}%")
    
    return numeric_columns_with_missing_values

# calculate statistics for both datasets
calculate_missing_values_numeric_columns(all_depression_train_cleaned, special_columns)
calculate_missing_values_numeric_columns(all_depression_test_cleaned, special_columns)

Number of numeric columns with missing values: 0
Mean of missing values per numeric column (in %): 0.00%
Standard deviation of missing values per numeric column (in %): 0.00%
Number of numeric columns with missing values: 0
Mean of missing values per numeric column (in %): 0.00%
Standard deviation of missing values per numeric column (in %): 0.00%


Index([], dtype='object')

In [125]:
# Function to impute missing numeric values with the mean
def impute_missing_values_mean(df, special_columns=None):
    if special_columns is None:
        special_columns = []
    
    # Select only the numeric columns for imputation, excluding special columns
    numeric_columns = df.select_dtypes(include=['number']).columns.difference(special_columns)
    
    # Impute missing values with the mean for each numeric column
    df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].mean())
    
    return df

# Apply the imputation function to both train and test datasets
all_depression_train_imputed = impute_missing_values_mean(all_depression_train_cleaned, special_columns)
all_depression_test_imputed = impute_missing_values_mean(all_depression_test_cleaned, special_columns)

## impute non-numeric missing values

In [126]:
# Function to remove duplicate columns in a DataFrame // special columns are accidentally by KNN imputer code duplicated
def remove_duplicate_columns(df, df_name):
    # Find and remove duplicated columns
    df_cleaned = df.loc[:, ~df.columns.duplicated()]
    print(f"Removed duplicate columns from {df_name}.")
    return df_cleaned

# Remove duplicate columns from each DataFrame
all_depression_train_imputed = remove_duplicate_columns(all_depression_train_imputed, 'all_depression_train_imputed')
all_depression_test_imputed = remove_duplicate_columns(all_depression_test_imputed, 'all_depression_test_imputed')

Removed duplicate columns from all_depression_train_imputed.
Removed duplicate columns from all_depression_test_imputed.


In [127]:
# Function to calculate missing value statistics for non-numeric columns only and print the results
def calculate_missing_values_non_numeric_columns(df, dataset_name, special_columns=None):
    if special_columns is None:
        special_columns = []
    
    # Select only the non-numeric columns for calculation, excluding special columns
    non_numeric_columns = df.select_dtypes(exclude=['number']).columns.difference(special_columns)
    
    # Total number of rows in the dataframe
    total_rows = len(df)
    
    # 1. Number of non-numeric columns with missing values
    non_numeric_columns_with_missing_values = non_numeric_columns[df[non_numeric_columns].isnull().sum() > 0]
    num_non_numeric_columns_with_missing_values = len(non_numeric_columns_with_missing_values)
    print(f"Number of non-numeric columns with missing values in {dataset_name}: {num_non_numeric_columns_with_missing_values}")
    
    # 2. Mean and standard deviation of missing values per non-numeric column (in percentage)
    missing_values_per_column_percent = df[non_numeric_columns].isnull().sum() / total_rows * 100  # Convert to percentage
    mean_missing_values_percent = missing_values_per_column_percent.mean()
    std_missing_values_percent = missing_values_per_column_percent.std()
    print(f"Mean of missing values per non-numeric column (in %) in {dataset_name}: {mean_missing_values_percent:.2f}%")
    print(f"Standard deviation of missing values per non-numeric column (in %) in {dataset_name}: {std_missing_values_percent:.2f}%")
    
# Apply the function to both train and test datasets
calculate_missing_values_non_numeric_columns(all_depression_train_imputed, "Train Dataset", special_columns)
calculate_missing_values_non_numeric_columns(all_depression_test_imputed, "Test Dataset", special_columns)

Number of non-numeric columns with missing values in Train Dataset: 0
Mean of missing values per non-numeric column (in %) in Train Dataset: 0.00%
Standard deviation of missing values per non-numeric column (in %) in Train Dataset: 0.00%
Number of non-numeric columns with missing values in Test Dataset: 0
Mean of missing values per non-numeric column (in %) in Test Dataset: 0.00%
Standard deviation of missing values per non-numeric column (in %) in Test Dataset: 0.00%


In [128]:
# Impute missing non-numeric values with the constant "missing" for all_depression_train_imputed, ignoring special_columns
for column in all_depression_train_imputed.select_dtypes(exclude=['number']).columns:
    if column not in special_columns:
        all_depression_train_imputed.loc[:, column] = all_depression_train_imputed[column].fillna('missing')

# Impute missing non-numeric values with the constant "missing" for all_depression_test_imputed, ignoring special_columns
for column in all_depression_test_imputed.select_dtypes(exclude=['number']).columns:
    if column not in special_columns:
        all_depression_test_imputed.loc[:, column] = all_depression_test_imputed[column].fillna('missing')

In [129]:
# Function to get data type counts for a DataFrame
def get_data_type_counts(df):
    # Get the data types for each column
    data_types = df.dtypes
    
    # Count the occurrences of each data type
    data_type_counts = data_types.value_counts()
    
    return data_type_counts

# Apply the function to all imputed datasets
data_type_counts_all_train = get_data_type_counts(all_depression_train_imputed)
data_type_counts_all_test = get_data_type_counts(all_depression_test_imputed)

# Display the counts for each dataset
print("\nData type counts for all_depression_train_imputed:")
print(data_type_counts_all_train)

print("\nData type counts for all_depression_test_imputed:")
print(data_type_counts_all_test)


Data type counts for all_depression_train_imputed:
object     485
float64    446
int64        1
dtype: int64

Data type counts for all_depression_test_imputed:
object            487
float64           446
datetime64[ns]      2
int64               1
dtype: int64


In [130]:
# Function to drop specified columns from a DataFrame
# those are columns that are accidentally taken from the UK Biobank database with datetime of specific baseline tests
# the column birth_date was created to calculate the age at baseline
def drop_columns(df, columns_to_drop):
    return df.drop(columns=columns_to_drop, errors='ignore')

# Columns to drop
columns_to_drop = ['p4286_i0', 'p4289_i0', 'birth_date']

# Apply the function to all imputed datasets
all_depression_train_imputed = drop_columns(all_depression_train_imputed, columns_to_drop)
all_depression_test_imputed = drop_columns(all_depression_test_imputed, columns_to_drop)

In [131]:
# Function to check for missing values in a DataFrame
def check_missing_values(df):
    # Check for missing values
    missing_values = df.isnull().sum()
    
    # Display columns with missing values (if any)
    missing_columns = missing_values[missing_values > 0]
    
    return missing_columns

# Apply the function to all four imputed datasets
missing_columns_all_train = check_missing_values(all_depression_train_imputed)
missing_columns_all_test = check_missing_values(all_depression_test_imputed)

# Print the results
print("\nMissing columns in all_depression_train_imputed:")
print(missing_columns_all_train)

print("\nMissing columns in all_depression_test_imputed:")
print(missing_columns_all_test)


Missing columns in all_depression_train_imputed:
Series([], dtype: int64)

Missing columns in all_depression_test_imputed:
Series([], dtype: int64)


# check class balance

In [132]:
# Function to count the number of 1s and 0s in the 'target' column
def count_target_values(df):
    count_ones = df['target'].sum()
    count_zeros = df['target'].count() - count_ones
    
    return count_ones, count_zeros

# Apply the function to all imputed datasets
count_ones_all_train, count_zeros_all_train = count_target_values(all_depression_train_imputed)
count_ones_all_test, count_zeros_all_test = count_target_values(all_depression_test_imputed)

# Print the results
print(f"\nNumber of 1s in all_depression_train_imputed: {count_ones_all_train}")
print(f"Number of 0s in all_depression_train_imputed: {count_zeros_all_train}")

print(f"\nNumber of 1s in all_depression_test_imputed: {count_ones_all_test}")
print(f"Number of 0s in all_depression_test_imputed: {count_zeros_all_test}")


Number of 1s in all_depression_train_imputed: 249
Number of 0s in all_depression_train_imputed: 251

Number of 1s in all_depression_test_imputed: 244
Number of 0s in all_depression_test_imputed: 256


# check for outliers

## check for numeric outliers

In [133]:
import pandas as pd

# Function to calculate range, IQR, and outlier percentage for each numeric feature and save to CSV
def generate_range_iqr_and_outlier_reports(df, file_range, file_iqr, file_outliers, special_columns=None):
    if special_columns is None:
        special_columns = []
    
    # Select only the numeric columns for calculation, excluding special columns
    numeric_columns = df.select_dtypes(include=['number']).columns.difference(special_columns)
    
    # Calculate Range (Max - Min) for each numeric column
    range_data = pd.DataFrame({
        'Feature': numeric_columns,
        'Range': df[numeric_columns].max() - df[numeric_columns].min()
    })
    # Sort by Range from most wide to most narrow
    range_data_sorted = range_data.sort_values(by='Range', ascending=False)
    
    # Save the Range report to CSV
    range_data_sorted.to_csv(file_range, index=False)
    print(f"Range report saved to {file_range}")
    
    # Calculate IQR (Q3 - Q1) for each numeric column
    iqr_data = pd.DataFrame({
        'Feature': numeric_columns,
        'IQR': df[numeric_columns].quantile(0.75) - df[numeric_columns].quantile(0.25)
    })
    # Sort by IQR from strongest to smallest outlier potential
    iqr_data_sorted = iqr_data.sort_values(by='IQR', ascending=False)
    
    # Save the IQR report to CSV
    iqr_data_sorted.to_csv(file_iqr, index=False)
    print(f"IQR report saved to {file_iqr}")
    
    # Calculate the percentage of outliers for each numeric column
    outlier_data = []
    
    for column in numeric_columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count outliers below and above the bounds
        below_outliers = (df[column] < lower_bound).sum()
        above_outliers = (df[column] > upper_bound).sum()
        
        total_rows = len(df)
        # Calculate percentage of outliers
        outlier_percentage = ((below_outliers + above_outliers) / total_rows) * 100
        
        outlier_data.append({
            'Feature': column,
            'Outlier Percentage': outlier_percentage
        })
    
    # Create a DataFrame from the outlier data
    outlier_data_df = pd.DataFrame(outlier_data)
    # Sort by the percentage of outliers from most to least
    outlier_data_sorted = outlier_data_df.sort_values(by='Outlier Percentage', ascending=False)
    
    # Save the Outliers report to CSV
    outlier_data_sorted.to_csv(file_outliers, index=False)
    print(f"Outliers report saved to {file_outliers}")

# File paths
file_range_csv = 'data/numeric_features_range.csv'
file_iqr_csv = 'data/numeric_features_iqr.csv'
file_outliers_csv = 'data/numeric_features_outliers.csv'

# Generate reports for range, IQR, and outliers
generate_range_iqr_and_outlier_reports(all_depression_train_imputed, file_range_csv, file_iqr_csv, file_outliers_csv)

Range report saved to data/numeric_features_range.csv
IQR report saved to data/numeric_features_iqr.csv
Outliers report saved to data/numeric_features_outliers.csv


In [134]:
def z_normalize(train_df, test_df, special_columns=None):
    if special_columns is None:
        special_columns = []

    # Add 'target' column to the list of special columns to be excluded from normalization
    columns_to_exclude = special_columns + ['target']

    # Filter out special columns and 'target' column from normalization in both train and test datasets
    special_columns_in_train = [col for col in columns_to_exclude if col in train_df.columns]
    special_columns_in_test = [col for col in columns_to_exclude if col in test_df.columns]
    
    # Select only the numeric columns for normalization, excluding special columns and 'target'
    numeric_columns_train = train_df.select_dtypes(include=['number']).columns.difference(special_columns_in_train)
    
    # Ensure the test dataframe has the same numeric columns in the same order as the train dataframe
    numeric_columns_test = numeric_columns_train.intersection(test_df.columns)
    
    # Initialize the StandardScaler
    scaler = StandardScaler()

    # Fit the scaler on the numeric training data and transform the training data
    train_df_scaled = train_df.copy()
    train_df_scaled[numeric_columns_train] = scaler.fit_transform(train_df[numeric_columns_train])
    
    # Transform the numeric test data using the scaler fitted on the training data
    test_df_scaled = test_df.copy()
    test_df_scaled[numeric_columns_test] = scaler.transform(test_df[numeric_columns_test])

    # Ensure non-numeric and special columns (including 'target') are retained in the test dataset
    train_df_scaled[special_columns_in_train] = train_df[special_columns_in_train]
    test_df_scaled[special_columns_in_test] = test_df[special_columns_in_test]
    
    return train_df_scaled, test_df_scaled

# Apply Z-normalization to the datasets using the function
all_depression_train_scaled, all_depression_test_scaled = z_normalize(all_depression_train_imputed, all_depression_test_imputed, special_columns)

# Function to check for missing values, excluding special columns and 'target'
def check_missing_values(df, special_columns=None):
    if special_columns is None:
        special_columns = []
    
    # Filter out special columns and 'target' column from the check
    columns_to_check = df.columns.difference(special_columns + ['target'])
    
    # Check for missing values
    missing_values = df[columns_to_check].isnull().sum()
    missing_values = missing_values[missing_values > 0]  # Filter only columns with missing values
    
    return missing_values

# Check for missing values in all datasets
missing_values_all_train = check_missing_values(all_depression_train_scaled, special_columns)
missing_values_all_test = check_missing_values(all_depression_test_scaled, special_columns)

print("\nMissing values in all_depression_train_scaled:")
print(missing_values_all_train)

print("\nMissing values in all_depression_test_scaled:")
print(missing_values_all_test)


Missing values in all_depression_train_scaled:
Series([], dtype: int64)

Missing values in all_depression_test_scaled:
Series([], dtype: int64)


## check for non-numeric outliers

In [135]:
# Function to count unique string occurrences in non-numeric columns and calculate mean and standard deviation
def count_unique_strings_with_stats(df, special_columns=None):
    if special_columns is None:
        special_columns = []
    
    unique_string_counts = []
    total_unique_count = 0

    # Filter out special columns that may no longer exist in the dataset
    special_columns_in_df = [col for col in special_columns if col in df.columns]
    
    # Iterate through each non-numeric column in the DataFrame, excluding special columns
    non_numeric_columns = df.select_dtypes(exclude=np.number).columns.difference(special_columns_in_df)
    
    for column in non_numeric_columns:
        # Get the counts of each unique value in the column
        value_counts = df[column].value_counts()
        
        # Count how many times a unique value occurs only once
        count_unique = (value_counts == 1).sum()
        
        # Store the count in the list and add to total
        unique_string_counts.append(count_unique)
        total_unique_count += count_unique

    # Calculate mean and standard deviation for the unique string counts
    mean_unique_count = np.mean(unique_string_counts)
    std_unique_count = np.std(unique_string_counts)

    # Count how many columns have at least one unique string occurrence
    columns_with_unique_entries = sum(count > 0 for count in unique_string_counts)
    
    return columns_with_unique_entries, total_unique_count, mean_unique_count, std_unique_count

# Apply the function to all scaled datasets using special_columns
unique_string_stats_train = count_unique_strings_with_stats(all_depression_train_scaled, special_columns)
unique_string_stats_test = count_unique_strings_with_stats(all_depression_test_scaled, special_columns)

# Adjusted print statements
datasets_info = [
    ("all_depression_train_scaled", unique_string_stats_train),
    ("all_depression_test_scaled", unique_string_stats_test)
]

for dataset_name, (columns_with_unique_entries, total_unique_count, mean_unique_count, std_unique_count) in datasets_info:
    print(f"Dataset '{dataset_name}' has {columns_with_unique_entries} columns with unique string entries.")
    print(f"The total number of unique string entries in '{dataset_name}' is {total_unique_count}.")
    print(f"Mean number of unique string entries per non-numeric feature in '{dataset_name}' is {mean_unique_count:.2f}.")
    print(f"Standard deviation of unique string entries per non-numeric feature in '{dataset_name}' is {std_unique_count:.2f}.\n")

Dataset 'all_depression_train_scaled' has 0 columns with unique string entries.
The total number of unique string entries in 'all_depression_train_scaled' is 0.
Mean number of unique string entries per non-numeric feature in 'all_depression_train_scaled' is 0.00.
Standard deviation of unique string entries per non-numeric feature in 'all_depression_train_scaled' is 0.00.

Dataset 'all_depression_test_scaled' has 0 columns with unique string entries.
The total number of unique string entries in 'all_depression_test_scaled' is 0.
Mean number of unique string entries per non-numeric feature in 'all_depression_test_scaled' is 0.00.
Standard deviation of unique string entries per non-numeric feature in 'all_depression_test_scaled' is 0.00.



### replace all non-numeric values that only occur once in a column with the most frequent value in this column for all columns

In [136]:
# Function to replace unique entries with the most frequent entry in non-numeric columns
def replace_unique_with_most_frequent(df, special_columns=None):
    if special_columns is None:
        special_columns = []
    
    # Filter out special columns that may no longer exist in the dataset
    special_columns_in_df = [col for col in special_columns if col in df.columns]
    
    # Iterate through each non-numeric column in the DataFrame, excluding special columns
    non_numeric_columns = df.select_dtypes(exclude=np.number).columns.difference(special_columns_in_df)
    
    for column in non_numeric_columns:
        # Get the value counts of the column
        value_counts = df[column].value_counts()
        
        # Identify the most frequent entry
        most_frequent_entry = value_counts.idxmax()
        
        # Replace all unique entries with the most frequent entry
        df[column] = df[column].apply(
            lambda x: most_frequent_entry if value_counts[x] == 1 else x
        )
    
    return df

# Apply the function to all scaled datasets excluding special_columns
all_depression_train_scaled = replace_unique_with_most_frequent(all_depression_train_scaled, special_columns)
all_depression_test_scaled = replace_unique_with_most_frequent(all_depression_test_scaled, special_columns)

## one hot encode all non-numeric columns

In [137]:
# Function to one-hot encode the training set and ensure the test set has the same columns
def one_hot_encode_train_test(train_df, test_df, special_columns=None, drop_first=False):
    if special_columns is None:
        special_columns = []
    
    # Filter out special columns that may no longer exist in the dataset
    special_columns_in_train = [col for col in special_columns if col in train_df.columns]
    special_columns_in_test = [col for col in special_columns if col in test_df.columns]
    
    # Separate the columns to exclude (special columns) from the rest of the DataFrame
    train_to_encode = train_df.drop(columns=special_columns_in_train)
    test_to_encode = test_df.drop(columns=special_columns_in_test)
    
    # One-hot encode the training set
    train_encoded = pd.get_dummies(train_to_encode, drop_first=drop_first)
    
    # One-hot encode the test set
    test_encoded = pd.get_dummies(test_to_encode, drop_first=drop_first)
    
    # Align the test set columns to match the training set columns
    test_encoded = test_encoded.reindex(columns=train_encoded.columns, fill_value=0)
    
    # Combine the encoded DataFrames with the special columns at the end
    train_final = pd.concat([train_encoded, train_df[special_columns_in_train]], axis=1)
    test_final = pd.concat([test_encoded, test_df[special_columns_in_test]], axis=1)
    
    return train_final, test_final

# Apply the function to both options (drop_first=False and drop_first=True)
all_depression_train_encoded, all_depression_test_encoded = one_hot_encode_train_test(
    all_depression_train_scaled, all_depression_test_scaled, special_columns, drop_first=False)

all_depression_train_encoded_drop, all_depression_test_encoded_drop = one_hot_encode_train_test(
    all_depression_train_scaled, all_depression_test_scaled, special_columns, drop_first=True)

# Print the head of each encoded DataFrame to verify the changes
print("\nall_depression_train_encoded head (drop_first=False):")
print(all_depression_train_encoded.head())

print("\nall_depression_test_encoded head (drop_first=False):")
print(all_depression_test_encoded.head())

print("\nall_depression_train_encoded_drop head (drop_first=True):")
print(all_depression_train_encoded_drop.head())

print("\nall_depression_test_encoded_drop head (drop_first=True):")
print(all_depression_test_encoded_drop.head())


all_depression_train_encoded head (drop_first=False):
          p34    p46_i0    p47_i0    p48_i0    p49_i0    p50_i0    p51_i0  \
126 -0.628473  0.863474 -0.039786  1.415492  0.650315  0.079338  0.292360   
314 -0.522276  1.593990 -1.079494 -1.273047 -1.216467 -1.596765  1.177561   
267  1.672459  0.323715 -0.994850 -1.431662 -1.561600  1.170085 -0.684851   
282  0.221102 -0.502104  1.059365 -1.196499  1.378711  1.261222  0.025512   
496 -1.194857  1.405302 -0.489943  1.290280  1.238361  1.614704 -1.639406   

       p68_i0    p74_i0    p77_i0  ...  p23075_i0_Category_A  \
126  0.932815  1.262318  1.308699  ...                     0   
314 -1.601980 -0.092063  1.037718  ...                     0   
267  0.670705 -0.518523  0.025388  ...                     0   
282 -1.183141  0.225843 -0.151887  ...                     0   
496  1.491095  0.251065 -1.756568  ...                     1   

     p23075_i0_Category_B  p23075_i0_Category_C  p23075_i0_Category_D  \
126                     

## clean column names to avoid problems in modelling

In [138]:
# Function to clean column names
def clean_column_name(name):
    # Replace spaces with underscores
    name = name.replace(' ', '_')
    # Remove any character that is not a letter, digit, or underscore
    name = re.sub(r'[^\w]', '', name)
    return name

# Function to clean column names in a DataFrame, excluding special columns
def clean_column_names(df, special_columns=None):
    if special_columns is None:
        special_columns = []
    
    # Filter out special columns that may no longer exist in the dataset
    special_columns_in_df = df.columns.intersection(special_columns)
    
    # Clean only the columns that are not in the special columns list
    columns_to_clean = df.columns.difference(special_columns_in_df)
    cleaned_columns = {col: clean_column_name(col) for col in columns_to_clean}
    
    # Rename the columns in the DataFrame
    df.rename(columns=cleaned_columns, inplace=True)
    
    return df

# Apply the function to all four encoded datasets
all_depression_train_encoded = clean_column_names(all_depression_train_encoded, special_columns)
all_depression_test_encoded = clean_column_names(all_depression_test_encoded, special_columns)
all_depression_train_encoded_drop = clean_column_names(all_depression_train_encoded_drop, special_columns)
all_depression_test_encoded_drop = clean_column_names(all_depression_test_encoded_drop, special_columns)

## set at the end all colums to datatype float 64 except target to int64

In [139]:
# Function to set 'target' column to int64 and all other columns to float64, excluding special columns
def set_column_types(df, target_column='target', special_columns=None):
    if special_columns is None:
        special_columns = []
    
    # Set the target column to int64
    if target_column in df.columns:
        df[target_column] = df[target_column].astype('int64')
    
    # Set all other columns to float64, excluding the target and special columns
    for column in df.columns:
        if column != target_column and column not in special_columns:
            df[column] = df[column].astype('float64')
    
    return df

# Apply the function to all four datasets
all_depression_train_encoded = set_column_types(all_depression_train_encoded, special_columns=special_columns)
all_depression_test_encoded = set_column_types(all_depression_test_encoded, special_columns=special_columns)
all_depression_train_encoded_drop = set_column_types(all_depression_train_encoded_drop, special_columns=special_columns)
all_depression_test_encoded_drop = set_column_types(all_depression_test_encoded_drop, special_columns=special_columns)

In [140]:
# Function to count the number of columns for each data type in a DataFrame
def count_datatype_columns(df):
    return df.dtypes.value_counts()

# Apply the function to all four datasets and store the results
datatype_counts_train_encoded = count_datatype_columns(all_depression_train_encoded)
datatype_counts_test_encoded = count_datatype_columns(all_depression_test_encoded)
datatype_counts_train_encoded_drop = count_datatype_columns(all_depression_train_encoded_drop)
datatype_counts_test_encoded_drop = count_datatype_columns(all_depression_test_encoded_drop)

# Display the counts for each dataset
print("Data type counts in all_depression_train_encoded (drop_first=False):")
print(datatype_counts_train_encoded)

print("\nData type counts in all_depression_test_encoded (drop_first=False):")
print(datatype_counts_test_encoded)

print("\nData type counts in all_depression_train_encoded_drop (drop_first=True):")
print(datatype_counts_train_encoded_drop)

print("\nData type counts in all_depression_test_encoded_drop (drop_first=True):")
print(datatype_counts_test_encoded_drop)

Data type counts in all_depression_train_encoded (drop_first=False):
float64    2856
int64         1
dtype: int64

Data type counts in all_depression_test_encoded (drop_first=False):
float64           2856
object               2
datetime64[ns]       2
int64                1
dtype: int64

Data type counts in all_depression_train_encoded_drop (drop_first=True):
float64    2374
int64         1
dtype: int64

Data type counts in all_depression_test_encoded_drop (drop_first=True):
float64           2374
object               2
datetime64[ns]       2
int64                1
dtype: int64


In [141]:
# Function to check for missing values in a DataFrame, excluding special columns
def check_missing_values(df, special_columns=None):
    if special_columns is None:
        special_columns = []
    
    # Filter out special columns that may no longer exist in the dataset
    columns_to_check = df.columns.difference(special_columns)
    
    # Check for missing values in the remaining columns
    missing_values = df[columns_to_check].isnull().sum()
    
    return missing_values[missing_values > 0]

# Apply the function to all four datasets and store the results
missing_values_train_encoded = check_missing_values(all_depression_train_encoded, special_columns)
missing_values_test_encoded = check_missing_values(all_depression_test_encoded, special_columns)
missing_values_train_encoded_drop = check_missing_values(all_depression_train_encoded_drop, special_columns)
missing_values_test_encoded_drop = check_missing_values(all_depression_test_encoded_drop, special_columns)

# Display the columns with missing values for each dataset
print("Columns with missing values in all_depression_train_encoded (drop_first=False):")
print(missing_values_train_encoded)

print("\nColumns with missing values in all_depression_test_encoded (drop_first=False):")
print(missing_values_test_encoded)

print("\nColumns with missing values in all_depression_train_encoded_drop (drop_first=True):")
print(missing_values_train_encoded_drop)

print("\nColumns with missing values in all_depression_test_encoded_drop (drop_first=True):")
print(missing_values_test_encoded_drop)

Columns with missing values in all_depression_train_encoded (drop_first=False):
Series([], dtype: int64)

Columns with missing values in all_depression_test_encoded (drop_first=False):
Series([], dtype: int64)

Columns with missing values in all_depression_train_encoded_drop (drop_first=True):
Series([], dtype: int64)

Columns with missing values in all_depression_test_encoded_drop (drop_first=True):
Series([], dtype: int64)


# store data to continue in a separate notebook

In [142]:
# Save each of the four datasets as a .pkl file in the "data" folder with the term "late" and the size appended
all_depression_train_encoded.to_pickle(f"data/all_late_depression_train_encoded_{all_depression_train_size}.pkl")
all_depression_test_encoded.to_pickle(f"data/all_late_depression_test_encoded_{all_depression_test_size}.pkl")
all_depression_train_encoded_drop.to_pickle(f"data/all_late_depression_train_encoded_drop_{all_depression_train_size}.pkl")
all_depression_test_encoded_drop.to_pickle(f"data/all_late_depression_test_encoded_drop_{all_depression_test_size}.pkl")